## Setup the notebook

In [ ]:
import os, sys

if os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
    from kaggle_secrets import UserSecretsClient
    import subprocess
    
    user_secrets = UserSecretsClient()
    token = user_secrets.get_secret("git_token")
    
    repo_url = f"git+https://{token}@github.com/Blaku03/PitchAnalyzer_AI.git"
    
    subprocess.check_call(
        ["pip", "install", repo_url],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.STDOUT
    )
    video_path = "/kaggle/input/pitchtest/B1606b0e6_1 (91).mp4"
    output_path = "./output_video.avi"
else:
    # reloads the code during development
    %load_ext autoreload
    %autoreload 2
    # ensure project root (one level up) is on PATH
    ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
    if ROOT not in sys.path:
        sys.path.insert(0, ROOT)

    video_path = "../sample_data/B1606b0e6_1 (91).mp4"
    output_path = "../output_videos/output_video.avi"
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

## Import the model

In [ ]:
from utils.import_models import ModelImporter
players_model = ModelImporter().download_player_model()

In [4]:
from trackers.game_tracker import GameTracker
from utils.video_utils import read_video, save_video
from utils.import_models import ModelImporter

players_model = ModelImporter().download_player_model()
video_path = "../sample_data/B1606b0e6_1 (91).mp4"
output_path = "../output_videos/output_video.avi"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

Model downloaded to /Users/bartekbrzyski/.cache/kagglehub/models/blaku03/player-detection/pyTorch/player_detectionv1_1/1


In [ ]:
from annotators.player_annotator import PlayersAnnotator
from trackers.game_tracker import GameTracker
from utils.video_utils import read_video
from utils.notebook_utils import show_image
import itertools

frame_generator = read_video(video_path)
first_frame = next(frame_generator)

tracks_generator =  GameTracker(players_model).get_detections_from_frames(frame_generator)
first_track = next(tracks_generator)

annotated_sample_image = PlayersAnnotator.annotate_frame(first_frame.copy(), first_track)
show_image(annotated_sample_image)

In [ ]:
from assigners.ball_statistics import BallStatictis

frame_generator = read_video(video_path)
tracks_generator =  PlayerDetector(players_model).get_detections_from_frames(frame_generator)

itr = 0
ball_stats = BallStatictis()
for track in tracks_generator:
    ball_stats.stats(track)
    itr += 1
    if itr == 10:
        break